State policies around the Covid-19 pandemic have varied significantly. In order to assess the relation of state policies to social media sentiment, we compiled a list of state policies and developed a cleaning method for validating cities and states in our dataset. 

In [1]:
import pandas as pd
import numpy as np
import datetime
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 60)

In [2]:
cities = pd.read_csv('./USA-cities-and-states/us_cities_states_counties.csv', sep = '|')

In [3]:
cities = cities.copy()

In [4]:
cities

,City,State short,State full,County,City alias
0,Holtsville,NY,New York,SUFFOLK,Internal Revenue Service
1,Holtsville,NY,New York,SUFFOLK,Holtsville
2,Adjuntas,PR,Puerto Rico,ADJUNTAS,URB San Joaquin
3,Adjuntas,PR,Puerto Rico,ADJUNTAS,Jard De Adjuntas
4,Adjuntas,PR,Puerto Rico,ADJUNTAS,Colinas Del Gigante
...,...,...,...,...,...
63206,Klawock,AK,Alaska,PRINCE OF WALES HYDER,Klawock
63207,Metlakatla,AK,Alaska,PRINCE OF WALES HYDER,Metlakatla
63208,Point Baker,AK,Alaska,PRINCE OF WALES HYDER,Point Baker
63209,Ward Cove,AK,Alaska,KETCHIKAN GATEWAY,Ward Cove


In [6]:
state = cities['State short'].unique()

In [7]:
state

array(['NY', 'PR', 'VI', 'MA', 'RI', 'NH', 'ME', 'VT', 'CT', 'NJ', 'AE',
       'PA', 'DE', 'DC', 'VA', 'MD', 'WV', 'NC', 'SC', 'GA', 'FL', nan,
       'AL', 'TN', 'MS', 'KY', 'OH', 'IN', 'MI', 'IA', 'WI', 'MN', 'SD',
       'ND', 'MT', 'IL', 'MO', 'KS', 'NE', 'LA', 'AR', 'OK', 'TX', 'CO',
       'WY', 'ID', 'UT', 'AZ', 'NM', 'NV', 'CA', 'AP', 'HI', 'AS', 'GU',
       'PW', 'FM', 'MP', 'MH', 'OR', 'WA', 'AK'], dtype=object)

The cities dataframe contains a list of every city in the US. In addition, the list contains military bases, cities in American territories such as Guam and Puerto Rico as well the countries Palau, Federated States of Micronesia and other countries in a 'free association' with the United States. For the purposes of policy assessment, we limited the tweet locations to the 50 states and the District of Columbia. This is partly due to the different legal relationships that territories and freely associated states have with the federal government. 

In [8]:
remove = {'MH', 'AE', 'FM', 'PW', 'AP', 'nan', 'PR', 'MP', 'GU', 'AS', 'PR', 'VI'}

In [9]:
type(remove)

set

In [10]:
clean_state = [s for s in state if str(s) not in remove]

In [11]:
len(clean_state)

51

In [12]:
clean_state

['NY',
 'MA',
 'RI',
 'NH',
 'ME',
 'VT',
 'CT',
 'NJ',
 'PA',
 'DE',
 'DC',
 'VA',
 'MD',
 'WV',
 'NC',
 'SC',
 'GA',
 'FL',
 'AL',
 'TN',
 'MS',
 'KY',
 'OH',
 'IN',
 'MI',
 'IA',
 'WI',
 'MN',
 'SD',
 'ND',
 'MT',
 'IL',
 'MO',
 'KS',
 'NE',
 'LA',
 'AR',
 'OK',
 'TX',
 'CO',
 'WY',
 'ID',
 'UT',
 'AZ',
 'NM',
 'NV',
 'CA',
 'HI',
 'OR',
 'WA',
 'AK']

Our clean_state list has been narrowed down to the 50 states plus DC. 

In [13]:
states = cities[['State short', 'State full']].drop_duplicates()

The states dataframe needs to be narrowed down because many cities have duplicated names for their common aliases. 

In [14]:
states

,State short,State full
0,NY,New York
2,PR,Puerto Rico
1869,VI,Virgin Islands
2696,MA,Massachusetts
3759,RI,Rhode Island
3900,NH,New Hampshire
4348,ME,Maine
5214,VT,Vermont
6097,CT,Connecticut
6682,NJ,New Jersey


In [15]:
states['validation'] = states['State short'].isin(clean_state)

In [16]:
states = states[states.validation == True].reset_index().drop(columns= 'index')

In [17]:
states

,State short,State full,validation
0,NY,New York,True
1,MA,Massachusetts,True
2,RI,Rhode Island,True
3,NH,New Hampshire,True
4,ME,Maine,True
5,VT,Vermont,True
6,CT,Connecticut,True
7,NJ,New Jersey,True
8,PA,Pennsylvania,True
9,DE,Delaware,True


Our states in the data frame are confirmed to line up with our master list. 

In [18]:
stay = pd.read_excel('./Stay-at-Home.xlsx', header = [2], nrows= 51)
stay.head()

,State,Date Announced,Effective Date
0,Alabama,April 3,April 4
1,Alaska,March 27,March 28
2,Arizona,March 30,March 31
3,Arkansas,-,-
4,California,March 19,March 19


In [19]:
# https://stackoverflow.com/questions/29298577/how-to-convert-string-to-datetime-with-nulls-python-pandas

form = '%B%d%Y'

stay['Date Announced'] = (stay['Date Announced'] + ' ' + '2020').str.replace(' ','')
stay['Date Announced'] = pd.to_datetime(stay['Date Announced'], format = form, errors='coerce')

stay['Effective Date'] = (stay['Effective Date'] + ' ' + '2020').str.replace(' ','')
stay['Effective Date'] = pd.to_datetime(stay['Effective Date'], format = form, errors='coerce')

The dates for stay at home orders had to be converted to datetime. 

In [20]:
stay

,State,Date Announced,Effective Date
0,Alabama,2020-04-03,2020-04-04
1,Alaska,2020-03-27,2020-03-28
2,Arizona,2020-03-30,2020-03-31
3,Arkansas,NaT,NaT
4,California,2020-03-19,2020-03-19
5,Colorado,2020-03-26,2020-03-26
6,Connecticut,2020-03-20,2020-03-23
7,Delaware,2020-03-22,2020-03-24
8,"Washington, D.C.",2020-03-30,2020-04-01
9,Florida,2020-04-01,2020-04-03


In [21]:
stay.dtypes

State                     object
Date Announced    datetime64[ns]
Effective Date    datetime64[ns]
dtype: object

In [22]:
states.rename(columns={'State short': 'state_short', 'State full': 'state_full'}, inplace= True)
states.head()

,state_short,state_full,validation
0,NY,New York,True
1,MA,Massachusetts,True
2,RI,Rhode Island,True
3,NH,New Hampshire,True
4,ME,Maine,True


In [23]:
stay.rename(columns={'State': 'state_full', 'Date Announced' : 'date_shelter_announced', 'Effective Date' : 'date_shelter_effective'}, inplace= True)
stay.head()

,state_full,date_shelter_announced,date_shelter_effective
0,Alabama,2020-04-03,2020-04-04
1,Alaska,2020-03-27,2020-03-28
2,Arizona,2020-03-30,2020-03-31
3,Arkansas,NaT,NaT
4,California,2020-03-19,2020-03-19


In [24]:
states = states.merge(stay, on = 'state_full', how = 'left')
states.head()

,state_short,state_full,validation,date_shelter_announced,date_shelter_effective
0,NY,New York,True,2020-03-20,2020-03-22
1,MA,Massachusetts,True,2020-03-23,2020-03-24
2,RI,Rhode Island,True,2020-03-28,2020-03-28
3,NH,New Hampshire,True,2020-03-26,2020-03-27
4,ME,Maine,True,2020-03-31,2020-04-02


In [25]:
kff = pd.read_excel('./7-30-2020_Social Distancing Order Effective and Rollback Dates.xlsx')

In [26]:
kff

,state_full,date_stay_home,date_travel_quarentine,date_nonessential_close,date_large_gather_ban,date_school_close,date_rest_limit,primary_delay,date_emergency_dec,date_end_stay_home,date_end_travel_quarentine,date_open_nonessential,date_end_large_gather_ban,date_end_school_close,date_end_rest_limit,Requirement Date,Masks Required?
0,Alabama,2020-04-04,NaT,2020-03-28,2020-03-19,2020-03-19,2020-03-17,NaN,2020-03-13,2020-04-30,NaT,2020-04-30,2020-05-11,NaT,2020-05-11,2020-07-16,Entire State
1,Alaska,2020-03-28,2020-03-25,2020-03-28,2020-03-24,2020-03-20,2020-03-18,NaN,2020-03-11,2020-04-24,NaT,2020-04-24,2020-04-24,NaT,2020-04-24,NaT,Parts of State
2,Arizona,2020-03-31,2020-04-09,2020-03-31,2020-03-30,2020-03-20,2020-03-20,NaN,2020-03-11,2020-05-16,2020-05-16,2020-05-08,2020-05-16,NaT,2020-05-11,NaT,Parts of State
3,Arkansas,NaT,2020-05-14,NaT,2020-03-27,2020-03-15,2020-03-19,NaN,2020-03-11,NaT,2020-06-15,NaT,2020-06-15,NaT,2020-05-11,2020-07-20,Entire State
4,California,2020-03-19,NaT,2020-03-19,2020-03-16,NaT,2020-03-16,NaN,2020-03-04,NaT,NaT,2020-05-08,NaT,NaT,NaT,2020-06-18,Entire State
5,Colorado,2020-03-26,NaT,2020-03-26,2020-03-26,2020-03-23,2020-03-17,NaN,2020-03-10,2020-04-27,NaT,2020-04-27,2020-04-27,NaT,2020-05-27,2020-07-17,Entire State
6,Connecticut,2020-03-23,2020-06-25,2020-03-23,2020-03-16,2020-03-17,2020-03-16,Postponed,2020-03-10,2020-05-20,NaT,2020-05-20,2020-06-01,NaT,2020-05-20,2020-04-17,Entire State
7,Delaware,2020-03-24,2020-03-30,2020-03-24,2020-03-16,2020-03-16,2020-03-16,Postponed,2020-03-12,2020-06-01,2020-06-01,2020-05-08,2020-06-15,NaT,2020-06-01,2020-05-01,Entire State
8,"Washington, D.C.",2020-04-01,NaT,2020-03-25,2020-03-20,2020-03-16,2020-03-16,NaN,2020-03-11,2020-05-29,NaT,2020-06-22,NaT,NaT,2020-05-29,2020-04-03,Entire Territory
9,Florida,2020-04-03,2020-03-24,2020-04-03,2020-03-25,2020-03-17,2020-03-20,NaN,2020-03-09,2020-05-04,NaT,2020-05-04,2020-06-05,NaT,2020-05-04,NaT,Parts of State


KFF had one of the most comprehensive datasets for state Covid-19 policies. The above information was mostly derived from KFF's [github](https://www.google.com) of raw data used in their website visualizations. Their visualizations can be viewed on KFF's [website](https://www.kff.org/policy-watch/stay-at-home-orders-to-fight-covid19/
) . 

The mask policy data was gathered from the organization [Masks4All](https://masks4all.co/what-states-require-masks/) , a volunteer organization tracking and promoting mask policies. 

In [27]:
states = states.merge(kff, on = 'state_full', how = 'left')
states.head()

,state_short,state_full,validation,date_shelter_announced,date_shelter_effective,date_stay_home,date_travel_quarentine,date_nonessential_close,date_large_gather_ban,date_school_close,date_rest_limit,primary_delay,date_emergency_dec,date_end_stay_home,date_end_travel_quarentine,date_open_nonessential,date_end_large_gather_ban,date_end_school_close,date_end_rest_limit,Requirement Date,Masks Required?
0,NY,New York,True,2020-03-20,2020-03-22,2020-03-22,2020-06-25,2020-03-22,2020-03-22,2020-03-20,2020-03-16,Canceled,2020-03-07,2020-06-27,NaT,2020-05-14,2020-05-22,NaT,2020-06-22,2020-04-15,Entire State
1,MA,Massachusetts,True,2020-03-23,2020-03-24,2020-03-22,2020-03-27,2020-03-24,2020-03-17,2020-03-16,2020-03-17,NaN,2020-03-10,2020-05-18,NaT,2020-05-19,NaT,NaT,2020-06-08,2020-05-06,Entire State
2,RI,Rhode Island,True,2020-03-28,2020-03-28,2020-03-28,2020-03-28,2020-03-30,2020-03-28,2020-03-23,2020-03-17,Postponed,2020-03-09,2020-05-09,NaT,2020-05-09,2020-06-01,NaT,2020-05-30,2020-04-20,Entire State
3,NH,New Hampshire,True,2020-03-26,2020-03-27,2020-03-27,NaT,2020-03-27,2020-03-16,2020-03-16,2020-03-16,NaN,2020-03-13,2020-06-15,NaT,2020-05-11,NaT,NaT,2020-05-18,NaT,Parts of State
4,ME,Maine,True,2020-03-31,2020-04-02,2020-04-01,2020-04-03,2020-03-25,2020-03-18,2020-04-02,2020-03-18,Postponed,2020-03-15,NaT,NaT,2020-05-01,2020-06-01,NaT,2020-06-01,2020-05-01,Entire State


In [28]:
states = states.rename(columns= {'Requirement Date' : 'date_mask_required', 'Masks Required?' : 'mask_required'  })

In [29]:
states.shape

(51, 21)

In [30]:
states.to_csv('state_policies_draft.csv', index= False)

After pulling in the dates from our various sources, the next step was to generate columns identifying whether each policy was enacted in a state in the first place. This was generated based on the presence or absence of dates for each policy after checking that missing dates were affiliated with lack of state policies. 

In [31]:
states.columns

Index(['state_short', 'state_full', 'validation', 'date_shelter_announced',
       'date_shelter_effective', 'date_stay_home', 'date_travel_quarentine',
       'date_nonessential_close', 'date_large_gather_ban', 'date_school_close',
       'date_rest_limit', 'primary_delay', 'date_emergency_dec',
       'date_end_stay_home', 'date_end_travel_quarentine',
       'date_open_nonessential', 'date_end_large_gather_ban',
       'date_end_school_close', 'date_end_rest_limit', 'date_mask_required',
       'mask_required'],
      dtype='object')

In [32]:
old_cols = [
       'date_shelter_effective', 'date_stay_home', 'date_travel_quarentine',
       'date_nonessential_close', 'date_large_gather_ban', 'date_school_close',
       'date_rest_limit', 'date_emergency_dec',
       'date_end_stay_home', 'date_end_travel_quarentine',
       'date_open_nonessential', 'date_end_large_gather_ban',
       'date_end_school_close', 'date_end_rest_limit'
]

In [33]:
new_cols = ['shelter_issued', 'stay_home_order', 'travel_quarentine_issued', 'nonessential_close_order', 
           'large_gather_ban_issued', 'school_closures', 'rest_limit_issued', 'emergency_declaration', 
            'end_stay_home', 'end_travel_quarentine', 'open_nonessential', 'end_large_gather_ban', 
           'schools_open_in_person', 'end_rest_limit']

In [34]:
old_cols[9], new_cols[9]

('date_end_travel_quarentine', 'end_travel_quarentine')

In [35]:
len(old_cols), len(new_cols)

(14, 14)

In [36]:
def policy_status(df, new_cols, old_cols):
    for ocol, ncol in zip(old_cols , new_cols):
        df[ncol] = states[ocol].notna().astype(int)
    return df

In [37]:
policy_status(states, new_cols, old_cols)

,state_short,state_full,validation,date_shelter_announced,date_shelter_effective,date_stay_home,date_travel_quarentine,date_nonessential_close,date_large_gather_ban,date_school_close,date_rest_limit,primary_delay,date_emergency_dec,date_end_stay_home,date_end_travel_quarentine,date_open_nonessential,date_end_large_gather_ban,date_end_school_close,date_end_rest_limit,date_mask_required,mask_required,shelter_issued,stay_home_order,travel_quarentine_issued,nonessential_close_order,large_gather_ban_issued,school_closures,rest_limit_issued,emergency_declaration,end_stay_home,end_travel_quarentine,open_nonessential,end_large_gather_ban,schools_open_in_person,end_rest_limit
0,NY,New York,True,2020-03-20,2020-03-22,2020-03-22,2020-06-25,2020-03-22,2020-03-22,2020-03-20,2020-03-16,Canceled,2020-03-07,2020-06-27,NaT,2020-05-14,2020-05-22,NaT,2020-06-22,2020-04-15,Entire State,1,1,1,1,1,1,1,1,1,0,1,1,0,1
1,MA,Massachusetts,True,2020-03-23,2020-03-24,2020-03-22,2020-03-27,2020-03-24,2020-03-17,2020-03-16,2020-03-17,NaN,2020-03-10,2020-05-18,NaT,2020-05-19,NaT,NaT,2020-06-08,2020-05-06,Entire State,1,1,1,1,1,1,1,1,1,0,1,0,0,1
2,RI,Rhode Island,True,2020-03-28,2020-03-28,2020-03-28,2020-03-28,2020-03-30,2020-03-28,2020-03-23,2020-03-17,Postponed,2020-03-09,2020-05-09,NaT,2020-05-09,2020-06-01,NaT,2020-05-30,2020-04-20,Entire State,1,1,1,1,1,1,1,1,1,0,1,1,0,1
3,NH,New Hampshire,True,2020-03-26,2020-03-27,2020-03-27,NaT,2020-03-27,2020-03-16,2020-03-16,2020-03-16,NaN,2020-03-13,2020-06-15,NaT,2020-05-11,NaT,NaT,2020-05-18,NaT,Parts of State,1,1,0,1,1,1,1,1,1,0,1,0,0,1
4,ME,Maine,True,2020-03-31,2020-04-02,2020-04-01,2020-04-03,2020-03-25,2020-03-18,2020-04-02,2020-03-18,Postponed,2020-03-15,NaT,NaT,2020-05-01,2020-06-01,NaT,2020-06-01,2020-05-01,Entire State,1,1,1,1,1,1,1,1,0,0,1,1,0,1
5,VT,Vermont,True,2020-03-24,2020-03-24,2020-03-24,2020-03-30,2020-03-25,2020-03-17,2020-03-18,2020-03-17,NaN,2020-03-13,2020-05-15,2020-06-08,2020-04-20,2020-06-01,NaT,2020-05-22,2020-08-01,Entire State,1,1,1,1,1,1,1,1,1,1,1,1,0,1
6,CT,Connecticut,True,2020-03-20,2020-03-23,2020-03-23,2020-06-25,2020-03-23,2020-03-16,2020-03-17,2020-03-16,Postponed,2020-03-10,2020-05-20,NaT,2020-05-20,2020-06-01,NaT,2020-05-20,2020-04-17,Entire State,1,1,1,1,1,1,1,1,1,0,1,1,0,1
7,NJ,New Jersey,True,2020-03-20,2020-03-21,2020-03-21,2020-06-25,2020-03-21,2020-03-16,2020-03-18,2020-03-16,Postponed,2020-03-09,2020-06-09,NaT,2020-05-18,2020-05-13,NaT,2020-06-15,2020-04-10,Entire State,1,1,1,1,1,1,1,1,1,0,1,1,0,1
8,PA,Pennsylvania,True,2020-03-23,2020-04-01,2020-04-01,NaT,2020-03-21,2020-04-01,2020-03-16,2020-03-17,NaN,2020-03-06,2020-05-08,NaT,2020-05-08,2020-05-08,NaT,2020-05-29,2020-04-17,Entire State,1,1,0,1,1,1,1,1,1,0,1,1,0,1
9,DE,Delaware,True,2020-03-22,2020-03-24,2020-03-24,2020-03-30,2020-03-24,2020-03-16,2020-03-16,2020-03-16,Postponed,2020-03-12,2020-06-01,2020-06-01,2020-05-08,2020-06-15,NaT,2020-06-01,2020-05-01,Entire State,1,1,1,1,1,1,1,1,1,1,1,1,0,1


In [39]:
states.drop(columns= ['date_shelter_announced', 'date_shelter_effective'], inplace= True)

Ultimately two columns for stay at home orders from earlier research were dropped because they were redundant with the shelter in place order dates. 

In [40]:
states.rename(columns= {'state_short' : 'state'}, inplace= True)

In [41]:
states.to_csv('state_policies_new.csv', index= False)

In [42]:
pd.read_csv('./state_policies_new.csv')

,state,state_full,validation,date_stay_home,date_travel_quarentine,date_nonessential_close,date_large_gather_ban,date_school_close,date_rest_limit,primary_delay,date_emergency_dec,date_end_stay_home,date_end_travel_quarentine,date_open_nonessential,date_end_large_gather_ban,date_end_school_close,date_end_rest_limit,date_mask_required,mask_required,shelter_issued,stay_home_order,travel_quarentine_issued,nonessential_close_order,large_gather_ban_issued,school_closures,rest_limit_issued,emergency_declaration,end_stay_home,end_travel_quarentine,open_nonessential,end_large_gather_ban,schools_open_in_person,end_rest_limit
0,NY,New York,True,2020-03-22,2020-06-25,2020-03-22,2020-03-22,2020-03-20,2020-03-16,Canceled,2020-03-07,2020-06-27,NaN,2020-05-14,2020-05-22,NaN,2020-06-22,2020-04-15,Entire State,1,1,1,1,1,1,1,1,1,0,1,1,0,1
1,MA,Massachusetts,True,2020-03-22,2020-03-27,2020-03-24,2020-03-17,2020-03-16,2020-03-17,NaN,2020-03-10,2020-05-18,NaN,2020-05-19,NaN,NaN,2020-06-08,2020-05-06,Entire State,1,1,1,1,1,1,1,1,1,0,1,0,0,1
2,RI,Rhode Island,True,2020-03-28,2020-03-28,2020-03-30,2020-03-28,2020-03-23,2020-03-17,Postponed,2020-03-09,2020-05-09,NaN,2020-05-09,2020-06-01,NaN,2020-05-30,2020-04-20,Entire State,1,1,1,1,1,1,1,1,1,0,1,1,0,1
3,NH,New Hampshire,True,2020-03-27,NaN,2020-03-27,2020-03-16,2020-03-16,2020-03-16,NaN,2020-03-13,2020-06-15,NaN,2020-05-11,NaN,NaN,2020-05-18,NaN,Parts of State,1,1,0,1,1,1,1,1,1,0,1,0,0,1
4,ME,Maine,True,2020-04-01,2020-04-03,2020-03-25,2020-03-18,2020-04-02,2020-03-18,Postponed,2020-03-15,NaN,NaN,2020-05-01,2020-06-01,NaN,2020-06-01,2020-05-01,Entire State,1,1,1,1,1,1,1,1,0,0,1,1,0,1
5,VT,Vermont,True,2020-03-24,2020-03-30,2020-03-25,2020-03-17,2020-03-18,2020-03-17,NaN,2020-03-13,2020-05-15,2020-06-08,2020-04-20,2020-06-01,NaN,2020-05-22,2020-08-01,Entire State,1,1,1,1,1,1,1,1,1,1,1,1,0,1
6,CT,Connecticut,True,2020-03-23,2020-06-25,2020-03-23,2020-03-16,2020-03-17,2020-03-16,Postponed,2020-03-10,2020-05-20,NaN,2020-05-20,2020-06-01,NaN,2020-05-20,2020-04-17,Entire State,1,1,1,1,1,1,1,1,1,0,1,1,0,1
7,NJ,New Jersey,True,2020-03-21,2020-06-25,2020-03-21,2020-03-16,2020-03-18,2020-03-16,Postponed,2020-03-09,2020-06-09,NaN,2020-05-18,2020-05-13,NaN,2020-06-15,2020-04-10,Entire State,1,1,1,1,1,1,1,1,1,0,1,1,0,1
8,PA,Pennsylvania,True,2020-04-01,NaN,2020-03-21,2020-04-01,2020-03-16,2020-03-17,NaN,2020-03-06,2020-05-08,NaN,2020-05-08,2020-05-08,NaN,2020-05-29,2020-04-17,Entire State,1,1,0,1,1,1,1,1,1,0,1,1,0,1
9,DE,Delaware,True,2020-03-24,2020-03-30,2020-03-24,2020-03-16,2020-03-16,2020-03-16,Postponed,2020-03-12,2020-06-01,2020-06-01,2020-05-08,2020-06-15,NaN,2020-06-01,2020-05-01,Entire State,1,1,1,1,1,1,1,1,1,1,1,1,0,1


In [43]:
tweets = pd.read_csv('./combined_us.csv')

C:\Users\ianbo\anaconda3\envs\dsi\lib\site-packages\IPython\core\interactiveshell.py:3072: DtypeWarning: Columns (19,20) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [44]:
tweet_policy = tweets.merge(states, on = 'state', how = 'left')
tweet_policy.head()

,created_at,favourites_count,status_id,lang,place_full_name,retweet_count,text,account_created_at,screen_name,followers_count,friends_count,verified,city,state,is_reply,is_retweet,country_code,place_type,user_id,source,is_quote,state_full,validation,date_stay_home,date_travel_quarentine,date_nonessential_close,date_large_gather_ban,date_school_close,date_rest_limit,primary_delay,date_emergency_dec,date_end_stay_home,date_end_travel_quarentine,date_open_nonessential,date_end_large_gather_ban,date_end_school_close,date_end_rest_limit,date_mask_required,mask_required,shelter_issued,stay_home_order,travel_quarentine_issued,nonessential_close_order,large_gather_ban_issued,school_closures,rest_limit_issued,emergency_declaration,end_stay_home,end_travel_quarentine,open_nonessential,end_large_gather_ban,schools_open_in_person,end_rest_limit
0,2020-08-10 04:44:47+00:00,0,1.292683e+18,en,"Dallas, TX",0,Matching our swimsuits may have been a step to...,2009-06-25 19:58:31+00:00,LoraleeL,1415,610,False,Dallas,TX,False,False,US,unspecified,NaN,NaN,NaN,Texas,True,2020-04-02,2020-03-30,2020-03-19,2020-03-19,2020-03-20,2020-03-20,NaN,2020-03-13,2020-04-30,2020-05-21,2020-05-01,2020-05-01,NaT,2020-05-01,2020-07-03,Entire State,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0
1,2020-08-10 06:18:56+00:00,0,1.292707e+18,en,"California, USA",0,1st staying overnight in a tent! Social distan...,2017-02-20 07:17:49+00:00,Lesmiz_79,48,569,False,unspecified,CA,False,False,US,unspecified,NaN,NaN,NaN,California,True,2020-03-19,NaT,2020-03-19,2020-03-16,NaT,2020-03-16,NaN,2020-03-04,NaT,NaT,2020-05-08,NaT,NaT,NaT,2020-06-18,Entire State,1.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
2,2020-08-10 04:59:59+00:00,0,1.292687e+18,en,"Jamesburg, NJ",0,Pandemic vibes✨ #friendsfromthemotherland #gua...,2011-06-21 15:43:45+00:00,MaruSardi10,51,241,False,Jamesburg,NJ,False,False,US,unspecified,NaN,NaN,NaN,New Jersey,True,2020-03-21,2020-06-25,2020-03-21,2020-03-16,2020-03-18,2020-03-16,Postponed,2020-03-09,2020-06-09,NaT,2020-05-18,2020-05-13,NaT,2020-06-15,2020-04-10,Entire State,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0
3,2020-08-10 04:56:50+00:00,0,1.292686e+18,en,"Baltimore, MD",0,I had this great conversation with my sister a...,2013-11-19 12:29:30+00:00,marloDeWawa,1001,890,False,Baltimore,MD,False,False,US,unspecified,NaN,NaN,NaN,Maryland,True,2020-03-30,2020-03-30,2020-03-23,2020-03-19,2020-03-16,2020-03-16,Postponed,2020-03-05,2020-05-15,NaT,2020-05-15,2020-06-10,NaT,2020-05-29,2020-04-18,Entire State,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0
4,2020-08-10 04:41:18+00:00,0,1.292682e+18,en,"Florida, USA",0,More Puzzles for your lockdown needs at WOD. @...,2012-03-10 04:36:48+00:00,IggyTorres,15,50,False,unspecified,FL,False,False,US,unspecified,NaN,NaN,NaN,Florida,True,2020-04-03,2020-03-24,2020-04-03,2020-03-25,2020-03-17,2020-03-20,NaN,2020-03-09,2020-05-04,NaT,2020-05-04,2020-06-05,NaT,2020-05-04,NaT,Parts of State,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0


In [45]:
tweet_policy.to_csv('policy_combined_us.csv', index= False)

Finally the policies were merged onto the overall set of tweets. 